<a href="https://colab.research.google.com/github/YifeiCAO/CogModelingRNNsTutorial/blob/main/Human_dataset_inner_outer_sub1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import and preparation

In [ ]:
!pip -q install -U "jax[tpu]" -f https://storage.googleapis.com/jax-releases/libtpu_releases.html

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 136.5/136.5 MB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.1/93.1 MB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 117.4 MB/s eta 0:00:00


In [ ]:
import requests
import pandas as pd
from io import StringIO
import gdown

import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

import numpy as np
import pandas as pd
# import plotnine as gg
# gg.theme_set(gg.theme_classic)  # for nicer-looking plots
import jax.numpy as jnp
import jax
import optax
import scipy

!pip install -U dm-haiku
import haiku as hk
rng_seq = hk.PRNGSequence(np.random.randint(2**32))

#@title Install required packages.
try:
    from google.colab import files
    _ON_COLAB = True
except:
    _ON_COLAB = False

if _ON_COLAB:
  !rm -rf CogModelingRNNsTutorial
  !git clone https://github.com/YifeiCAO/CogModelingRNNsTutorial
  !pip install -e CogModelingRNNsTutorial/CogModelingRNNsTutorial
  !cp CogModelingRNNsTutorial/CogModelingRNNsTutorial/*py CogModelingRNNsTutorial
else:
  !pip install CogModelingRNNsTutorial/requirements.txt

#@title Imports + defaults settings.
%load_ext autoreload
%autoreload 2
# for reload
# %reload_ext autoreload

# import haiku as hk
# import jax
# import jax.numpy as jnp
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import optax
import os
import warnings

# warnings.filterwarnings("ignore")

# try:
#     from google.colab import files
#     _ON_COLAB = True
# except:
#     _ON_COLAB = False

from CogModelingRNNsTutorial import bandits
from CogModelingRNNsTutorial import disrnn
from CogModelingRNNsTutorial import hybrnn
from CogModelingRNNsTutorial import hybconrnn
from CogModelingRNNsTutorial import hybrnn_direct_con
from CogModelingRNNsTutorial import plotting
from CogModelingRNNsTutorial import rat_data
from CogModelingRNNsTutorial import rnn_utils

from google.colab import drive
import pickle

# 挂载 Google Drive
drive.mount('/content/drive')

/usr/local/lib/python3.11/dist-packages/jax/_src/cloud_tpu_init.py:84: UserWarning: Transparent hugepages are not enabled. TPU runtime startup and shutdown time should be significantly improved on TPU v5e and newer. If not already set, you may need to enable transparent hugepages in your VM image (sudo sh -c "echo always > /sys/kernel/mm/transparent_hugepage/enabled")
  warnings.warn(


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 373.8/373.8 kB 15.0 MB/s eta 0:00:00
Cloning into 'CogModelingRNNsTutorial'...
remote: Enumerating objects: 1274, done.
remote: Counting objects: 100% (162/162), done.
remote: Compressing objects: 100% (24/24), done.
remote: Total 1274 (delta 147), reused 139 (delta 138), pack-reused 1112 (from 1)
Receiving objects: 100% (1274/1274), 6.57 MiB | 12.54 MiB/s, done.
Resolving deltas: 100% (820/820), done.
Obtaining file:///content/CogModelingRNNsTutorial/CogModelingRNNsTutorial
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for CogModelingRNNsTutorial (pyproject.toml) ... done
  Created wheel for CogModelingRNNsTutorial: filename=cogmodelingrnnstutorial-0.0.0-0.editable-py3-none-any.whl size=3095 sha256=a1be8333362e65a6da2fad6f91e1095f2fe4f22e2eeeca6b536548c119cfacf5


In [ ]:
def compute_negative_log_likelihood(dataset, model_fun, params):
    xs, actual_choices = next(dataset)
    n_trials_per_session, n_sessions = actual_choices.shape[:2]

    # 直接在外部运行 eval_model（不要放 jit 里！）
    model_outputs, _ = rnn_utils.eval_model(model_fun, params, xs)

    # 再 jit 一个小函数，只对 model_outputs 做 softmax
    @jax.jit
    def compute_log_probs(logits):
        return jax.nn.log_softmax(logits[:, :, :2], axis=-1)

    predicted_log_choice_probabilities = compute_log_probs(model_outputs)

    # 后续仍转为 NumPy 做索引
    predicted_log_choice_probabilities = np.array(predicted_log_choice_probabilities)

    total_log_likelihood = 0.0
    total_valid_trials = 0

    for sess_i in range(n_sessions):
        for trial_i in range(n_trials_per_session):
            actual_choice = int(actual_choices[trial_i, sess_i])
            if actual_choice >= 0:
                total_log_likelihood += predicted_log_choice_probabilities[trial_i, sess_i, actual_choice]
                total_valid_trials += 1

    if total_valid_trials == 0:
        raise ValueError("No valid trials found.")

    avg_nll = -total_log_likelihood / total_valid_trials
    return avg_nll



def evaluate_nll_over_full_dataset(dataset, model_fun, params):
    all_nlls = []
    for _ in range(dataset.n_batches):
        nll = compute_negative_log_likelihood(dataset, model_fun, params)
        all_nlls.append(nll)

    avg_nll = np.mean(all_nlls)
    print(f"[All Batches Averaged] NLL: {avg_nll:.4f}")
    return avg_nll


In [ ]:
@jax.jit
def compute_log_likelihood(dataset, model_fun, params):
    xs, actual_choices = next(dataset)
    n_trials_per_session, n_sessions = actual_choices.shape[:2]
    model_outputs, model_states = rnn_utils.eval_model(model_fun, params, xs)

    # predicted log-probs for the first two actions
    predicted_log_choice_probabilities = np.array(
        jax.nn.log_softmax(model_outputs[:, :, :2], axis=-1)
    )

    n_actions = predicted_log_choice_probabilities.shape[2]
    log_likelihoods = []

    for sess_i in range(n_sessions):
        log_likelihood = 0.0
        n = 0
        for trial_i in range(n_trials_per_session):
            actual_choice = int(actual_choices[trial_i, sess_i])
            # ignore invalid trials (<0 or ≥n_actions)
            if 0 <= actual_choice < n_actions:
                log_likelihood += predicted_log_choice_probabilities[
                    trial_i, sess_i, actual_choice
                ]
                n += 1

        if n > 0:
            normalized_likelihood = np.exp(log_likelihood / n)
            log_likelihoods.append(normalized_likelihood)

    mean_likelihood = np.mean(log_likelihoods)
    std_likelihood  = np.std(log_likelihoods)

    print(f'Average Normalized Likelihood: {100 * mean_likelihood:.1f}%')
    return mean_likelihood, std_likelihood


In [ ]:
jax.devices()

[TpuDevice(id=0, process_index=0, coords=(0,0,0), core_on_chip=0)]

## Human Datas

In [ ]:
osf_url = 'https://osf.io/download/xe6yu/'
response = requests.get(osf_url)

# Check the request
if response.status_code == 200:
    # Read as pandas dataframe
    qasim_data = pd.read_csv(StringIO(response.text))
    print('File downloaded and read successfully!')
else:
    print('Failed to download file. Status code:', response.status_code)

qasim_data.head()

# # read data
selected_columns = ['participant', 'trials_gamble', 'gamble', 'prob', 'reward']

qasim = qasim_data[selected_columns]
qasim_filtered = qasim[qasim['trials_gamble'].notna()]
qasim_sorted = qasim_filtered.groupby('participant', group_keys=False).apply(lambda x: x.sort_values('trials_gamble'))
qasim_sorted = qasim_sorted.reset_index(drop=True)
qasim_sorted['participant'] = qasim_sorted.groupby(['participant']).ngroup() + 1
qasim_sorted['action'] = qasim_sorted['gamble']
qasim_sorted

# —— 2) 把缺失的 action 填成 -1 —— #
qasim_sorted['action'] = qasim_sorted['action'].fillna(-1).astype(int)

# —— 3) 如果需要，把 reward 映射到 0/1 —— #
# （如果已经是 0/1 可跳过；否则取消下面注释并调整映射字典）
# qasim_sorted['reward'] = (
#     qasim_sorted['reward']
#     .map({-1: 0, 1: 1})
#     .fillna(-1)
#     .astype(int)
# )

# —— 4) 排序，确保 trial 顺序 —— #
qasim_sorted = qasim_sorted.sort_values(
    ['participant', 'trials_gamble']
).reset_index(drop=True)

# —— 5) 生成下一步动作 action_n —— #
qasim_sorted['action_n'] = (
    qasim_sorted
    .groupby('participant')['action']
    .shift(-1)
)
# 每个 participant 最后一 trial 的 action_n 设为 -1
last_idxs = qasim_sorted.groupby('participant').tail(1).index
qasim_sorted.loc[last_idxs, 'action_n'] = -1

# —— 6) 按 participant 构造 xs_list, ys_list —— #
xs_list, ys_list = [], []
for pid, grp in qasim_sorted.groupby('participant'):
    grp = grp.sort_values('trials_gamble')
    x = grp[['action', 'reward']].to_numpy().astype(float)    # 输入特征
    y = grp[['action_n']].to_numpy().astype(int)             # 下一步动作
    xs_list.append(x)
    ys_list.append(y)

# —— 7) 堆成三维数组 —— #
xs_qa = np.stack(xs_list, axis=1)  # (n_sessions, n_trials, 2)
ys_qa = np.stack(ys_list, axis=1)  # (n_sessions, n_trials, 1)

print("xs.shape =", xs_qa.shape)
print("ys.shape =", ys_qa.shape)



File downloaded and read successfully!
xs.shape = (60, 206, 2)
ys.shape = (60, 206, 1)


### Sidarus dataset

In [ ]:
# # 修改后的 file_id
# file_id = '1TSV6CdyClKz831qD2ln4z3WjLLcOgG1n'
# download_url = f'https://drive.google.com/uc?id={file_id}'

# # 下载并保存为 'downloaded_file.csv'
# output_file = 'downloaded_file.csv'
# gdown.download(download_url, output_file, quiet=False)

# # 读取 CSV 数据
# sida_data = pd.read_csv(output_file)
# sida_data

# # 1) action 从 {1,2} → {0,1}，并把所有缺失值填成 -1
# sida_data['action'] = (
#     sida_data['action']
#     .map({1: 0, 2: 1})    # 映射
#     .fillna(-1)           # 缺失值设为 -1
#     .astype(int)
# )

# # 2) outcome 从 {-1,1} → {0,1}，缺失也填成 -1
# sida_data['reward'] = (
#     sida_data['outcome']
#     .map({-1: 0, 1: 1})
#     .fillna(-1)
#     .astype(int)
# )

# # 3) 给每个被试×session 分配一个连续的 session_id
# sida_data['session_id'] = (
#     sida_data
#     .groupby(['subj', 'session'], sort=False)
#     .ngroup()
#     + 1
# )

# # 4) 排序，保证 trial 顺序
# sida_data = sida_data.sort_values(
#     ['session_id', 'epN', 'epTrialN']
# ).reset_index(drop=True)

# # 5) 生成下一步动作 action_n；每个 session 末尾设为 -1
# sida_data['action_n'] = (
#     sida_data
#     .groupby('session_id')['action']
#     .shift(-1)
# )
# last_idx = sida_data.groupby('session_id').tail(1).index
# sida_data.loc[last_idx, 'action_n'] = -1

# # 6) 按 session_id 抽取序列，并堆成 xs, ys
# xs_list, ys_list = [], []
# for sid, grp in sida_data.groupby('session_id'):
#     grp = grp.sort_values(['epN', 'epTrialN'])
#     x = grp[['action', 'reward']].to_numpy().astype(float)  # 特征
#     y = grp[['action_n']].to_numpy().astype(int)              # Label
#     xs_list.append(x)
#     ys_list.append(y)

# # 最终结果：xs.shape == (n_sessions, n_trials, 2)，ys.shape == (n_sessions, n_trials, 1)
# xs_sida = np.stack(xs_list, axis=1)
# ys_sida = np.stack(ys_list, axis=1)

# print("xs.shape =", xs_sida.shape)
# print("ys.shape =", ys_sida.shape)


### Schaaf dataset

In [ ]:
# 修改后的 file_id
file_id = '1rJFmDhCE3fdXtSHSSvtre_7V49gGsg7P'
download_url = f'https://drive.google.com/uc?id={file_id}'

# 下载并保存为 'downloaded_file.csv'
output_file = 'downloaded_file.csv'
gdown.download(download_url, output_file, quiet=False)

# 读取 CSV 数据
schaaf_data = pd.read_csv(output_file)
schaaf_data

df1 = schaaf_data[['pp', 'trial1', 'response1', 'outcome1']].copy()
df1.columns = ['pp', 'trial_in_session', 'response', 'reward']
df1['session'] = 1

df2 = schaaf_data[['pp', 'trial2', 'response2', 'outcome2']].copy()
df2.columns = ['pp', 'trial_in_session', 'response', 'reward']
df2['session'] = 2

# —— 上面拆分、concat、sort 的部分保持不变 —— #

# 合并成长表
df_long = pd.concat([df1, df2], ignore_index=True)
df_long = df_long.sort_values(['pp', 'session', 'trial_in_session']).reset_index(drop=True)

# —— 映射 outcome，再填 missing response —— #
# 把原来的 response1/response2 改名后是 `response`
# 把 outcome1/outcome2 改名后是 `reward`
# 把 outcome 映射成 1/0，然后把原本缺失的 reward 补成 -1
df_long['reward'] = (
    df_long['reward']
    .map({1: 1, -1: 0})
    .fillna(-1)
    .astype(int)
)

# 把缺失的 response 一样补成 -1
df_long['response'] = (
    df_long['response']
    .fillna(-1)
    .astype(int)
)


# 接下来再做 session_id、shift action_n、以及 stack xs/ys 的流程……
df_long['session_id'] = (df_long['pp'] - 1) * 2 + df_long['session']
df_long['action_n'] = df_long.groupby('session_id')['response'].shift(-1)
last_idx = df_long.groupby('session_id').tail(1).index
df_long.loc[last_idx, 'action_n'] = -1

# 生成 xs, ys
session_ids = df_long['session_id'].unique()
xs_list, ys_list = [], []
for sid in session_ids:
    sd = df_long[df_long['session_id'] == sid].sort_values('trial_in_session')
    x = sd[['response', 'reward']].to_numpy().astype(float)
    y = sd[['action_n']].to_numpy().astype(int)
    xs_list.append(x)
    ys_list.append(y)

xs = np.stack(xs_list, axis=0)   # (n_sessions, n_trials, 2)
ys = np.stack(ys_list, axis=0)   # (n_sessions, n_trials, 1)


# 或者第 0 维是 trials，第 1 维是 sessions：
xs_sch = np.stack(xs_list, axis=1)  # (n_trials, n_sessions, 2)
ys_sch = np.stack(ys_list, axis=1)  # (n_trials, n_sessions, 1)

print("xs.shape =", xs_sch.shape)
print("ys.shape =", ys_sch.shape)

# 1 → 0
# 方法一：一次性用 np.where 嵌套
xs_sch[:, :, 0] = np.where(
    xs_sch[:, :, 0] == 2,  # 如果等于 2
    1,                         # 则变成 1
    np.where(
        xs_sch[:, :, 0] == 1,  # 否则如果等于 1
        0,                         # 则变成 0
        xs_sch[:, :, 0]        # 其它值保持原样
    )
)
ys_sch[ys_sch == 1] = 0
ys_sch[ys_sch == 2] = 1

Downloading...
From: https://drive.google.com/uc?id=1rJFmDhCE3fdXtSHSSvtre_7V49gGsg7P
To: /content/downloaded_file.csv
100%|██████████| 374k/374k [00:00<00:00, 7.64MB/s]

xs.shape = (249, 94, 2)
ys.shape = (249, 94, 1)


In [ ]:
# xs_flat = xs_sch[:-1, :, 1]  # 或用 ys[:,:,0]
# # 1) 有效 trial 的掩码（0 或 1 才算），shape (n_trials, n_sessions)
# valid = xs_flat >= 0
# # 2) 每个 session 成功的次数（sum of 1s）
# n_success = np.sum(xs_flat * valid, axis=0)
# # 3) 每个 session 的有效 trial 数
# n_valid   = np.sum(valid, axis=0)
# # 4) 正确率 = 成功次数 / 有效数
# accuracy_per_session = n_success / n_valid

# # 打印
# for i, acc in enumerate(accuracy_per_session):
#     print(f"Session {i:02d}: accuracy = {acc:.4f} ({acc*100:.2f}%)")

### Maria Dataset

In [ ]:
import pandas as pd
import numpy as np
import gdown

# —— 1) 下载并读入原始数据 —— #
file_id = '1N_zAy-qrbfjvF8Kbb504IH2JNhR5KI-P'
url     = f'https://drive.google.com/uc?id={file_id}'
gdown.download(url, 'downloaded_file.csv', quiet=False)
eck_data = pd.read_csv('downloaded_file.csv')

# —— 2) 筛选＆重命名列 —— #
sel = ['sID','TrialID','selected_box','reward']
eck_sorted = eck_data[sel].copy()
eck_sorted['participant'] = eck_sorted.groupby('sID').ngroup()+1
eck_sorted['action']      = eck_sorted['selected_box']

# 只保留至少做够 120 试次的被试
max_trial = eck_sorted.groupby('participant')['TrialID'].transform('max')
eck_sorted = eck_sorted[max_trial>=120]

# 只用前 120 试次
eck_sorted = eck_sorted[eck_sorted['TrialID']<=120].reset_index(drop=True)

# —— 3) 生成下一步动作 action_n —— #
def generate_action_n(group):
    group = group.sort_values('TrialID')
    group['action_n'] = group['action'].shift(-1).fillna(-1).astype(int)
    return group

eck_sorted = (
    eck_sorted
    .groupby('participant', group_keys=False)
    .apply(generate_action_n)
    .reset_index(drop=True)
)

# —— 4) 按 participant 构造 xs_list/ys_list —— #
xs_list, ys_list = [], []
for pid, grp in eck_sorted.groupby('participant'):
    grp = grp.sort_values('TrialID').iloc[:120]
    x = grp[['action','reward']].to_numpy().astype(float)  # (120,2)
    y = grp[['action_n']].to_numpy().astype(int)           # (120,1)
    xs_list.append(x)
    ys_list.append(y)

# —— 5) stack 成 (n_sessions, n_trials, feat_dim) —— #
xs_ma = np.stack(xs_list, axis=1)  # (305,120,2)
ys_ma = np.stack(ys_list, axis=1)  # (305,120,1)

print("xs_ma.shape =", xs_ma.shape)
print("ys_ma.shape =", ys_ma.shape)


Downloading...
From: https://drive.google.com/uc?id=1N_zAy-qrbfjvF8Kbb504IH2JNhR5KI-P
To: /content/downloaded_file.csv
100%|██████████| 2.57M/2.57M [00:00<00:00, 17.0MB/s]


xs_ma.shape = (120, 305, 2)
ys_ma.shape = (120, 305, 1)


## Clean Data

## Generate a big human dataset with all experiments, session length is 130 trial per session

In [ ]:
import numpy as np

def segment_and_pad(x: np.ndarray,
                    y: np.ndarray,
                    seg_len: int = 130,
                    pad_x: float = 0.,
                    pad_y: int = -1):
    T, D = x.shape
    n_segs = int(np.ceil(T / seg_len))
    x_segs, y_segs = [], []
    for i in range(n_segs):
        start = i * seg_len
        end = start + seg_len
        x_part = x[start : min(end, T)]
        y_part = y[start : min(end, T)]
        pad = end - min(end, T)
        if pad > 0:
            x_part = np.pad(x_part,
                            pad_width=((0, pad), (0, 0)),
                            constant_values=pad_x)
            y_part = np.pad(y_part,
                            pad_width=((0, pad), (0, 0)),
                            constant_values=pad_y)
        x_segs.append(x_part)
        y_segs.append(y_part)
    return x_segs, y_segs

# —— 假设你已经有这几组 (xs, ys) —— #
# xs_qa   (60,  206, 2), ys_qa  (60,206, 1)
# xs_sida (800, 40,  2), ys_sida(800,40, 1)
# xs_sch  (249, 44,  2), ys_sch (249,44, 1)
# xs_ma   (120,305, 2), ys_ma  (120,305,1)

all_xs, all_ys = [], []

for xs, ys in [(xs_qa, ys_qa),
               (xs_sch, ys_sch),
               (xs_ma, ys_ma)]:

    # 如果是 (n_trials, n_sessions, feat) 维度，就直接：
    # T, N, D = xs.shape
    # 否则若是 (n_sessions, n_trials, feat)，先转：
    # xs = xs.transpose(1,0,2)
    # ys = ys.transpose(1,0,2)

    T, N, D = xs.shape
    for sess in range(N):
        x_seq = xs[:, sess, :]  # (T, D)
        y_seq = ys[:, sess, :]  # (T, 1)
        x_segs, y_segs = segment_and_pad(
            x_seq, y_seq,
            seg_len=130,
            pad_x=0., pad_y=-1
        )
        all_xs.extend(x_segs)
        all_ys.extend(y_segs)

# —— 修改在这里：不要 np.stack，直接输出列表 —— #
# all_xs 是一个 Python list，长度 = 总片段数，每个元素 shape=(130, D)
# all_ys 是一个 Python list，长度 = 总片段数，每个元素 shape=(130, 1)

print(f"Total segments: {len(all_xs)}")
print(f"First xs segment shape: {all_xs[0].shape}")
print(f"First ys segment shape: {all_ys[0].shape}")

# 如果你需要把它们返回成变量：
xs_segment_list = all_xs
ys_segment_list = all_ys


Total segments: 699
First xs segment shape: (130, 2)
First ys segment shape: (130, 1)


In [ ]:
def format_into_datasets_10_multi(
    xs_list: list[np.ndarray],
    ys_list: list[np.ndarray],
    dataset_constructor,
    batch_size: int = None,
    random_seed: int = None,
):
    """
    对多只猴子独立做 10‐fold CV，每折：
      - test = 每只猴子第 i 份 fold
      - validation 抽取与 test 大小一致的 session
      - 剩余做 train
    Args:
      xs_list: list of np.ndarray, 每项形状 (timesteps, n_sess_i, feat)
      ys_list: list of np.ndarray, 每项形状 (timesteps, n_sess_i, ...)
      dataset_constructor: 接受 (xs_sub, ys_sub, batch_size) 的构造器
      batch_size: mini‐batch 大小；None 则每 dataset 用全部 episodes
      random_seed: int，可复现打乱
    Returns:
      List of 10 tuples (ds_train, ds_val, ds_test)
    """
    rng = np.random.RandomState(random_seed) if random_seed is not None else np.random

    n_monkeys = len(xs_list)
    # 1. 对每只猴子生成打乱后的 session 索引，并切成 10 份
    folds_per_monkey = []
    for xs in xs_list:
        n_sess = xs.shape[1]
        idx = rng.permutation(n_sess)
        folds = np.array_split(idx, 10)
        folds_per_monkey.append(folds)

    folds_datasets = []
    for i in range(10):
        # 2a. 测试集：三猴子各自的第 i 份
        test_parts = [folds_per_monkey[m][i] for m in range(n_monkeys)]

        # 2b. 剩余 sessions 用于 train+val
        rem_parts = [
            np.hstack([folds_per_monkey[m][j] for j in range(10) if j != i])
            for m in range(n_monkeys)
        ]

        # 2c. 验证集大小与测试集一致：每只猴子抽取与 test_parts[m] 相同数量
        val_parts = []
        train_parts = []
        for m in range(n_monkeys):
            rng.shuffle(rem_parts[m])
            n_test = len(test_parts[m])
            val = rem_parts[m][:n_test]
            train = rem_parts[m][n_test:]
            val_parts.append(val)
            train_parts.append(train)

        # 3. 拼接各猴子的 train/val/test xs, ys
        def _gather(parts_idx):
            xs_sub, ys_sub = [], []
            for m, idxs in enumerate(parts_idx):
                xs_sub.append(xs_list[m][:, idxs, :])
                ys_sub.append(ys_list[m][:, idxs, :])
            return np.concatenate(xs_sub, axis=1), np.concatenate(ys_sub, axis=1)

        xs_train, ys_train = _gather(train_parts)
        xs_val,   ys_val   = _gather(val_parts)
        xs_test,  ys_test  = _gather(test_parts)

        # 4. 构造 DatasetRNN
        ds_train = dataset_constructor(xs_train, ys_train, batch_size=batch_size)
        ds_val   = dataset_constructor(xs_val,   ys_val,   batch_size=batch_size)
        ds_test  = dataset_constructor(xs_test,  ys_test,  batch_size=batch_size)

        folds_datasets.append((ds_train, ds_val, ds_test))

    return folds_datasets


In [ ]:
# 假设你已经有原始的：
#   xs_qa   (T_qa,   N_qa,   D),   ys_qa   (T_qa,   N_qa,   1)
#   xs_sida (T_sida, N_sida, D),   ys_sida (T_sida, N_sida, 1)
#   xs_sch  (T_sch,  N_sch,  D),   ys_sch  (T_sch,  N_sch,  1)
#   xs_ma   (T_ma,   N_ma,   D),   ys_ma   (T_ma,   N_ma,   1)

def make_segmented_array(xs, ys, seg_len=130, pad_x=0., pad_y=-1):
    all_xs, all_ys = [], []
    T, N, D = xs.shape
    for sess in range(N):
        x_seq = xs[:, sess, :]    # (T, D)
        y_seq = ys[:, sess, :]    # (T, 1)
        x_segs, y_segs = segment_and_pad(x_seq, y_seq, seg_len, pad_x, pad_y)
        all_xs.extend(x_segs)     # list of (130, D)
        all_ys.extend(y_segs)     # list of (130, 1)
    # 把 list 再拼成一个三维 array (130, n_segments, D)
    xs_seg = np.stack(all_xs, axis=1)
    ys_seg = np.stack(all_ys, axis=1)
    return xs_seg, ys_seg

# 针对四个源分别做一次
xs_qa_seg,   ys_qa_seg   = make_segmented_array(xs_qa,   ys_qa)
xs_sch_seg, ys_sch_seg  = make_segmented_array(xs_sch,  ys_sch)
xs_ma_seg,  ys_ma_seg   = make_segmented_array(xs_ma,   ys_ma)

In [ ]:
xs_human_list = [xs_qa_seg, xs_sch_seg, xs_ma_seg]
ys_human_list = [ys_qa_seg, ys_sch_seg, ys_ma_seg]

# Fitting different models

## Vanilla RNN inner-outer loop

In [ ]:
AS_OUTER_LOOP = [4, 5] # assigned outer loop(s) to run in this notebook

In [ ]:
import numpy as np
import itertools
import os
import pandas as pd
import time
from sklearn.model_selection import KFold

# =====================================
# Step 0: 合并所有实验到 session 维
# =====================================
# 假设 xs_human_list / ys_human_list: list of 3 arrays
# 每个数组形状: (n_trials, n_sessions_exp, feature_dim)
xs_all = np.concatenate(xs_human_list, axis=1)  # 在 session 维拼接
ys_all = np.concatenate(ys_human_list, axis=1)  # 同样在 session 维拼接

n_trials, total_sessions, feature_dim = xs_all.shape
_, _, target_dim = ys_all.shape

print(f"总 trials: {n_trials}, 总 sessions: {total_sessions}")
print(f"X shape: {xs_all.shape}, Y shape: {ys_all.shape}")

# =====================================
# Step 1: 按索引切分 dataset
# =====================================
def format_into_datasets_by_index(xs,
                                  ys,
                                  dataset_constructor,
                                  train_idx,
                                  val_idx,
                                  test_idx,
                                  batch_size=None):
    ds_train = dataset_constructor(xs[:, train_idx], ys[:, train_idx], batch_size=batch_size)
    ds_val   = dataset_constructor(xs[:, val_idx], ys[:, val_idx], batch_size=batch_size)
    ds_test  = dataset_constructor(xs[:, test_idx], ys[:, test_idx], batch_size=batch_size)
    return ds_train, ds_val, ds_test

# =====================================
# Step 2: 超参数网格
# =====================================
param_grid = {
    'hidden_size': [8, 16, 32, 64],
    'lr': [1e-3, 1e-4],
    'weight_decay': [1e-3, 1e-4],
    'batch_size': [32, 64],
}
K_outer = 10
K_inner = 3
random_seed = 42

# =====================================
# Step 3: 结果保存路径
# =====================================
OUTPUT_DIR = '/content/drive/MyDrive/Results'
os.makedirs(OUTPUT_DIR, exist_ok=True)
inner_csv = os.path.join(OUTPUT_DIR, 'human_vrnn_inner_search_results_sub1.csv')
outer_csv = os.path.join(OUTPUT_DIR, 'human_vrnn_outer_results_sub1.csv')

if os.path.exists(inner_csv):
    df_inner_existing = pd.read_csv(inner_csv)
else:
    df_inner_existing = pd.DataFrame(columns=[
        'outer_fold', 'hidden_size', 'lr', 'weight_decay', 'batch_size', 'avg_val_nll', 'time'
    ])
if os.path.exists(outer_csv):
    df_outer_existing = pd.read_csv(outer_csv)
else:
    df_outer_existing = pd.DataFrame(columns=[
        'outer_fold', 'best_hidden_size', 'best_lr', 'best_weight_decay', 'best_batch_size', 'test_nll'
    ])

# =====================================
# Step 4: 外层 CV
# =====================================
outer_cv = KFold(n_splits=K_outer, shuffle=True, random_state=random_seed)
session_indices = np.arange(total_sessions)

for outer_fold, (train_val_idx, test_idx) in enumerate(outer_cv.split(session_indices), start=1):

    print(f"\n=== Outer Fold {outer_fold}/{K_outer} ===")

    if outer_fold not in AS_OUTER_LOOP:
        print(f"Skipping outer fold {outer_fold} (not assigned in this notebook)")
        # print('train_val_idx: ', train_val_idx)
        print('test_idx: ', test_idx)
        continue

    if (df_outer_existing['outer_fold'] == outer_fold).any():
        print(f"Skipping outer fold {outer_fold} (already done)")
        continue

    print('test_idx: ', test_idx)
    # ==================
    # 内层超参搜索
    # ==================
    inner_cv = KFold(n_splits=K_inner, shuffle=True, random_state=random_seed)
    best_score = float('inf')
    best_params = None

    for hs, lr, wd, bs in itertools.product(
            param_grid['hidden_size'],
            param_grid['lr'],
            param_grid['weight_decay'],
            param_grid['batch_size']):

        mask = (
            (df_inner_existing['outer_fold'] == outer_fold) &
            (df_inner_existing['hidden_size'] == hs) &
            (df_inner_existing['lr'] == lr) &
            (df_inner_existing['weight_decay'] == wd) &
            (df_inner_existing['batch_size'] == bs)
        )
        if mask.any():
            print(f"Skipping inner config fold {outer_fold}: hs={hs}, lr={lr}, wd={wd}, bs={bs}")
            continue

        inner_scores = []
        t0 = time.time()

        for inner_train_idx, inner_val_idx in inner_cv.split(train_val_idx):



            train_sessions = train_val_idx[inner_train_idx]
            val_sessions = train_val_idx[inner_val_idx]

            ds_train, ds_val, _ = format_into_datasets_by_index(
                xs_all, ys_all,
                rnn_utils.DatasetRNN,
                train_idx=train_sessions,
                val_idx=val_sessions,
                test_idx=[],
                batch_size=bs
            )

            def make_rnn():
                return hk.DeepRNN([hk.VanillaRNN(hs), hk.Linear(output_size=2)])

            params, _ = rnn_utils.fit_model(
                model_fun=make_rnn,
                dataset_train=ds_train,
                dataset_test=ds_val,
                loss_fun='categorical',
                optimizer=optax.chain(
                    optax.add_decayed_weights(wd),
                    optax.adam(learning_rate=lr)
                ),
                n_steps_per_call=1000,
                n_steps_max=100000,
                early_stop_step=200,
                if_early_stop=True
            )

            avg_nll = evaluate_nll_over_full_dataset(ds_val, make_rnn, params)
            inner_scores.append(avg_nll)

        mean_inner_score = np.mean(inner_scores)
        df_inner_existing = pd.concat([
            df_inner_existing,
            pd.DataFrame([{
                'outer_fold': outer_fold,
                'hidden_size': hs, 'lr': lr, 'weight_decay': wd, 'batch_size': bs,
                'avg_val_nll': mean_inner_score,
                'time': time.time() - t0
            }])
        ], ignore_index=True)
        df_inner_existing.to_csv(inner_csv, index=False)

        if mean_inner_score < best_score:
            best_score = mean_inner_score
            best_params = {'hidden_size': hs, 'lr': lr, 'weight_decay': wd, 'batch_size': bs}

    print(f"Best params for outer fold {outer_fold}: {best_params}, val NLL={best_score:.4f}")

    # ==================
    # 外层训练 + 测试
    # ==================
    ds_train_val, _, ds_test = format_into_datasets_by_index(
        xs_all, ys_all,
        rnn_utils.DatasetRNN,
        train_idx=train_val_idx,
        val_idx=[],
        test_idx=test_idx,
        batch_size=best_params['batch_size']
    )

    def make_best_rnn():
        return hk.DeepRNN([hk.VanillaRNN(best_params['hidden_size']), hk.Linear(output_size=2)])

    params, _ = rnn_utils.fit_model(
        model_fun=make_best_rnn,
        dataset_train=ds_train_val,
        dataset_test=ds_test,
        loss_fun='categorical',
        optimizer=optax.chain(
            optax.add_decayed_weights(best_params['weight_decay']),
            optax.adam(learning_rate=best_params['lr'])
        ),
        n_steps_per_call=1000,
        n_steps_max=100000,
        early_stop_step=200,
        if_early_stop=True
    )

    test_nll = evaluate_nll_over_full_dataset(ds_test, make_best_rnn, params)
    print(f"Outer fold {outer_fold} test NLL: {test_nll:.4f}")

    df_outer_existing = pd.concat([
        df_outer_existing,
        pd.DataFrame([{
            'outer_fold': outer_fold,
            'best_hidden_size': best_params['hidden_size'],
            'best_lr': best_params['lr'],
            'best_weight_decay': best_params['weight_decay'],
            'best_batch_size': best_params['batch_size'],
            'test_nll': test_nll
        }])
    ], ignore_index=True)
    df_outer_existing.to_csv(outer_csv, index=False)

# =====================================
# Step 5: 最终结果
# =====================================
avg_nll = df_outer_existing['test_nll'].mean()
se_nll = df_outer_existing['test_nll'].std(ddof=1) / np.sqrt(len(df_outer_existing))
print(f"\nFinal Average NLL: {avg_nll:.4f} ± {se_nll:.4f}")


Streaming output truncated to the last 5000 lines.
Step 1000 of 1000; Loss: 1.5983e+03; Test Loss: 8.1644e+02. (Time: 2.5s)
first loss: inf test loss new: 3825.204250488281
updating best model ..
Step 1000 of 100000; Loss: 3.8252e+03. (Time: 0.0s)
Step 1000 of 1000; Loss: 1.5644e+03; Test Loss: 7.9952e+02. (Time: 2.6s)
first loss: 3825.204250488281 test loss new: 2483.0555395507813
updating best model ..
Step 2000 of 100000; Loss: 2.4831e+03. (Time: 0.0s)
Step 1000 of 1000; Loss: 1.5560e+03; Test Loss: 7.9657e+02. (Time: 2.6s)
first loss: 2483.0555395507813 test loss new: 2466.6158190917968
updating best model ..
Step 3000 of 100000; Loss: 2.4666e+03. (Time: 0.0s)
Step 1000 of 1000; Loss: 1.5438e+03; Test Loss: 7.9263e+02. (Time: 2.6s)
first loss: 2466.6158190917968 test loss new: 2457.8440478515627
updating best model ..
Step 4000 of 100000; Loss: 2.4578e+03. (Time: 0.0s)
Step 1000 of 1000; Loss: 1.5165e+03; Test Loss: 7.7983e+02. (Time: 2.6s)
first loss: 2457.8440478515627 test loss 

## RL-ANN inner-outer

In [ ]:
AS_OUTER_LOOP = [1,2,3,4] # assigned outer loop(s) to run in this notebook

In [ ]:
import numpy as np
import itertools
import os
import pandas as pd
import time
from sklearn.model_selection import KFold

# =====================================
# Step 0: 合并所有实验到 session 维
# =====================================
# 假设 xs_human_list / ys_human_list: list of 3 arrays
# 每个数组形状: (n_trials, n_sessions_exp, feature_dim)
xs_all = np.concatenate(xs_human_list, axis=1)  # 在 session 维拼接
ys_all = np.concatenate(ys_human_list, axis=1)  # 同样在 session 维拼接

n_trials, total_sessions, feature_dim = xs_all.shape
_, _, target_dim = ys_all.shape

print(f"总 trials: {n_trials}, 总 sessions: {total_sessions}")
print(f"X shape: {xs_all.shape}, Y shape: {ys_all.shape}")

# =====================================
# Step 1: 按索引切分 dataset
# =====================================
def format_into_datasets_by_index(xs,
                                  ys,
                                  dataset_constructor,
                                  train_idx,
                                  val_idx,
                                  test_idx,
                                  batch_size=None):
    ds_train = dataset_constructor(xs[:, train_idx], ys[:, train_idx], batch_size=batch_size)
    ds_val   = dataset_constructor(xs[:, val_idx], ys[:, val_idx], batch_size=batch_size)
    ds_test  = dataset_constructor(xs[:, test_idx], ys[:, test_idx], batch_size=batch_size)
    return ds_train, ds_val, ds_test

# =====================================
# Step 2: 超参数网格
# =====================================
param_grid = {
    'hidden_size': [8, 16, 32, 64],
    'lr': [1e-3, 1e-4],
    'weight_decay': [1e-3, 1e-4],
    'batch_size': [32, 64],
}
K_outer = 10
K_inner = 3
random_seed = 42

# =====================================
# Step 3: 结果保存路径
# =====================================
OUTPUT_DIR = '/content/drive/MyDrive/Results'
os.makedirs(OUTPUT_DIR, exist_ok=True)
inner_csv = os.path.join(OUTPUT_DIR, 'human_rlann_inner_search_results_sub1.csv')
outer_csv = os.path.join(OUTPUT_DIR, 'human_rlann_outer_results_sub1.csv')

if os.path.exists(inner_csv):
    df_inner_existing = pd.read_csv(inner_csv)
else:
    df_inner_existing = pd.DataFrame(columns=[
        'outer_fold', 'hidden_size', 'lr', 'weight_decay', 'batch_size', 'avg_val_nll', 'time'
    ])
if os.path.exists(outer_csv):
    df_outer_existing = pd.read_csv(outer_csv)
else:
    df_outer_existing = pd.DataFrame(columns=[
        'outer_fold', 'best_hidden_size', 'best_lr', 'best_weight_decay', 'best_batch_size', 'test_nll'
    ])

# =====================================
# Step 4: 外层 CV
# =====================================
outer_cv = KFold(n_splits=K_outer, shuffle=True, random_state=random_seed)
session_indices = np.arange(total_sessions)

for outer_fold, (train_val_idx, test_idx) in enumerate(outer_cv.split(session_indices), start=1):

    print(f"\n=== Outer Fold {outer_fold}/{K_outer} ===")

    if outer_fold not in AS_OUTER_LOOP:
        print(f"Skipping outer fold {outer_fold} (not assigned in this notebook)")
        # print('train_val_idx: ', train_val_idx)
        print('test_idx: ', test_idx)
        continue

    if (df_outer_existing['outer_fold'] == outer_fold).any():
        print(f"Skipping outer fold {outer_fold} (already done)")
        continue

    print('test_idx: ', test_idx)
    # ==================
    # 内层超参搜索
    # ==================
    inner_cv = KFold(n_splits=K_inner, shuffle=True, random_state=random_seed)
    best_score = float('inf')
    best_params = None

    for hs, lr, wd, bs in itertools.product(
            param_grid['hidden_size'],
            param_grid['lr'],
            param_grid['weight_decay'],
            param_grid['batch_size']):

        mask = (
            (df_inner_existing['outer_fold'] == outer_fold) &
            (df_inner_existing['hidden_size'] == hs) &
            (df_inner_existing['lr'] == lr) &
            (df_inner_existing['weight_decay'] == wd) &
            (df_inner_existing['batch_size'] == bs)
        )
        if mask.any():
            print(f"Skipping inner config fold {outer_fold}: hs={hs}, lr={lr}, wd={wd}, bs={bs}")
            continue

        inner_scores = []
        t0 = time.time()

        for inner_train_idx, inner_val_idx in inner_cv.split(train_val_idx):



            train_sessions = train_val_idx[inner_train_idx]
            val_sessions = train_val_idx[inner_val_idx]

            ds_train, ds_val, _ = format_into_datasets_by_index(
                xs_all, ys_all,
                rnn_utils.DatasetRNN,
                train_idx=train_sessions,
                val_idx=val_sessions,
                test_idx=[],
                batch_size=bs
            )

            use_hidden_state = 'False'
            use_previous_values = 'False'
            fit_forget = "False"
            habit_weight = 1

            value_weight = 1.  # This is needed for it to be doing RL

            rnn_rl_params = {
                's': use_hidden_state == 'True',
                'o': use_previous_values == 'True',
                'fit_forget': fit_forget == 'True',
                'forget': 0.,
                'w_h': habit_weight,
                'w_v': value_weight}
            network_params = {'n_actions': 2, 'hidden_size': hs}

            def make_rlann():
              model = hybrnn.BiRNN(rl_params=rnn_rl_params, network_params=network_params)
              return model

            params, _ = rnn_utils.fit_model(
                model_fun=make_rlann,
                dataset_train=ds_train,
                dataset_test=ds_val,
                loss_fun='categorical',
                optimizer=optax.chain(
                    optax.add_decayed_weights(wd),
                    optax.adam(learning_rate=lr)
                ),
                n_steps_per_call=1000,
                n_steps_max=100000,
                early_stop_step=200,
                if_early_stop=True
            )

            avg_nll = evaluate_nll_over_full_dataset(ds_val, make_rlann, params)
            inner_scores.append(avg_nll)

        mean_inner_score = np.mean(inner_scores)
        df_inner_existing = pd.concat([
            df_inner_existing,
            pd.DataFrame([{
                'outer_fold': outer_fold,
                'hidden_size': hs, 'lr': lr, 'weight_decay': wd, 'batch_size': bs,
                'avg_val_nll': mean_inner_score,
                'time': time.time() - t0
            }])
        ], ignore_index=True)
        df_inner_existing.to_csv(inner_csv, index=False)

        if mean_inner_score < best_score:
            best_score = mean_inner_score
            best_params = {'hidden_size': hs, 'lr': lr, 'weight_decay': wd, 'batch_size': bs}

    print(f"Best params for outer fold {outer_fold}: {best_params}, val NLL={best_score:.4f}")

    # ==================
    # 外层训练 + 测试
    # ==================
    ds_train_val, _, ds_test = format_into_datasets_by_index(
        xs_all, ys_all,
        rnn_utils.DatasetRNN,
        train_idx=train_val_idx,
        val_idx=[],
        test_idx=test_idx,
        batch_size=best_params['batch_size']
    )

    use_hidden_state = 'False'
    use_previous_values = 'False'
    fit_forget = "False"
    habit_weight = 1

    value_weight = 1.  # This is needed for it to be doing RL

    rnn_rl_params = {
        's': use_hidden_state == 'True',
        'o': use_previous_values == 'True',
        'fit_forget': fit_forget == 'True',
        'forget': 0.,
        'w_h': habit_weight,
        'w_v': value_weight}
    network_params = {'n_actions': 2, 'hidden_size': best_params['hidden_size']}

    def make_best_rlann():
      model = hybrnn.BiRNN(rl_params=rnn_rl_params, network_params=network_params)
      return model

    params, _ = rnn_utils.fit_model(
        model_fun=make_best_rlann,
        dataset_train=ds_train_val,
        dataset_test=ds_test,
        loss_fun='categorical',
        optimizer=optax.chain(
            optax.add_decayed_weights(best_params['weight_decay']),
            optax.adam(learning_rate=best_params['lr'])
        ),
        n_steps_per_call=1000,
        n_steps_max=100000,
        early_stop_step=200,
        if_early_stop=True
    )

    test_nll = evaluate_nll_over_full_dataset(ds_test, make_best_rlann, params)
    print(f"Outer fold {outer_fold} test NLL: {test_nll:.4f}")

    df_outer_existing = pd.concat([
        df_outer_existing,
        pd.DataFrame([{
            'outer_fold': outer_fold,
            'best_hidden_size': best_params['hidden_size'],
            'best_lr': best_params['lr'],
            'best_weight_decay': best_params['weight_decay'],
            'best_batch_size': best_params['batch_size'],
            'test_nll': test_nll
        }])
    ], ignore_index=True)
    df_outer_existing.to_csv(outer_csv, index=False)

# =====================================
# Step 5: 最终结果
# =====================================
avg_nll = df_outer_existing['test_nll'].mean()
se_nll = df_outer_existing['test_nll'].std(ddof=1) / np.sqrt(len(df_outer_existing))
print(f"\nFinal Average NLL: {avg_nll:.4f} ± {se_nll:.4f}")


Streaming output truncated to the last 5000 lines.
Step 1000 of 1000; Loss: 8.7878e+02; Test Loss: 1.6047e+03. (Time: 3.2s)
first loss: 1339.8238916015625 test loss new: 1329.2968359375
updating best model ..
Step 8000 of 100000; Loss: 1.3293e+03. (Time: 0.0s)
Step 1000 of 1000; Loss: 1.9135e+03; Test Loss: 1.6795e+03. (Time: 3.2s)
first loss: 1329.2968359375 test loss new: 1322.6567150878907
updating best model ..
Step 9000 of 100000; Loss: 1.3227e+03. (Time: 0.0s)
Step 1000 of 1000; Loss: 8.7897e+02; Test Loss: 1.8686e+03. (Time: 3.1s)
first loss: 1322.6567150878907 test loss new: 1335.8343432617187

Stopping early as the loss at step 200 did not improve over step 1.
[All Batches Averaged] NLL: 0.4393
Step 1000 of 1000; Loss: 2.2614e+03; Test Loss: 2.2439e+03. (Time: 3.1s)
first loss: inf test loss new: 3377.822717285156
updating best model ..
Step 1000 of 100000; Loss: 3.3778e+03. (Time: 0.0s)
Step 1000 of 1000; Loss: 1.0673e+03; Test Loss: 2.2057e+03. (Time: 3.2s)
first loss: 3377.

In [ ]:
import numpy as np
import itertools
import os
import pandas as pd
import time
from sklearn.model_selection import KFold

# =====================================
# Step 0: 合并所有实验到 session 维
# =====================================
# 假设 xs_human_list / ys_human_list: list of 3 arrays
# 每个数组形状: (n_trials, n_sessions_exp, feature_dim)
xs_all = np.concatenate(xs_human_list, axis=1)  # 在 session 维拼接
ys_all = np.concatenate(ys_human_list, axis=1)  # 同样在 session 维拼接

n_trials, total_sessions, feature_dim = xs_all.shape
_, _, target_dim = ys_all.shape

print(f"总 trials: {n_trials}, 总 sessions: {total_sessions}")
print(f"X shape: {xs_all.shape}, Y shape: {ys_all.shape}")

# =====================================
# Step 1: 按索引切分 dataset
# =====================================
def format_into_datasets_by_index(xs,
                                  ys,
                                  dataset_constructor,
                                  train_idx,
                                  val_idx,
                                  test_idx,
                                  batch_size=None):
    ds_train = dataset_constructor(xs[:, train_idx], ys[:, train_idx], batch_size=batch_size)
    ds_val   = dataset_constructor(xs[:, val_idx], ys[:, val_idx], batch_size=batch_size)
    ds_test  = dataset_constructor(xs[:, test_idx], ys[:, test_idx], batch_size=batch_size)
    return ds_train, ds_val, ds_test

# =====================================
# Step 2: 超参数网格
# =====================================
param_grid = {
    'hidden_size': [8, 16, 32, 64],
    'lr': [1e-3, 1e-4],
    'weight_decay': [1e-3, 1e-4],
    'batch_size': [32, 64],
}
K_outer = 10
K_inner = 3
random_seed = 42

# =====================================
# Step 3: 结果保存路径
# =====================================
OUTPUT_DIR = '/content/drive/MyDrive/Results'
os.makedirs(OUTPUT_DIR, exist_ok=True)
inner_csv = os.path.join(OUTPUT_DIR, 'human_conann_inner_search_results_sub1.csv')
outer_csv = os.path.join(OUTPUT_DIR, 'human_conann_outer_results_sub1.csv')

if os.path.exists(inner_csv):
    df_inner_existing = pd.read_csv(inner_csv)
else:
    df_inner_existing = pd.DataFrame(columns=[
        'outer_fold', 'hidden_size', 'lr', 'weight_decay', 'batch_size', 'avg_val_nll', 'time'
    ])
if os.path.exists(outer_csv):
    df_outer_existing = pd.read_csv(outer_csv)
else:
    df_outer_existing = pd.DataFrame(columns=[
        'outer_fold', 'best_hidden_size', 'best_lr', 'best_weight_decay', 'best_batch_size', 'test_nll'
    ])

# =====================================
# Step 4: 外层 CV
# =====================================
outer_cv = KFold(n_splits=K_outer, shuffle=True, random_state=random_seed)
session_indices = np.arange(total_sessions)

for outer_fold, (train_val_idx, test_idx) in enumerate(outer_cv.split(session_indices), start=1):

    print(f"\n=== Outer Fold {outer_fold}/{K_outer} ===")

    if outer_fold not in AS_OUTER_LOOP:
        print(f"Skipping outer fold {outer_fold} (not assigned in this notebook)")
        # print('train_val_idx: ', train_val_idx)
        print('test_idx: ', test_idx)
        continue

    if (df_outer_existing['outer_fold'] == outer_fold).any():
        print(f"Skipping outer fold {outer_fold} (already done)")
        continue

    print('test_idx: ', test_idx)
    # ==================
    # 内层超参搜索
    # ==================
    inner_cv = KFold(n_splits=K_inner, shuffle=True, random_state=random_seed)
    best_score = float('inf')
    best_params = None

    for hs, lr, wd, bs in itertools.product(
            param_grid['hidden_size'],
            param_grid['lr'],
            param_grid['weight_decay'],
            param_grid['batch_size']):

        mask = (
            (df_inner_existing['outer_fold'] == outer_fold) &
            (df_inner_existing['hidden_size'] == hs) &
            (df_inner_existing['lr'] == lr) &
            (df_inner_existing['weight_decay'] == wd) &
            (df_inner_existing['batch_size'] == bs)
        )
        if mask.any():
            print(f"Skipping inner config fold {outer_fold}: hs={hs}, lr={lr}, wd={wd}, bs={bs}")
            continue

        inner_scores = []
        t0 = time.time()

        for inner_train_idx, inner_val_idx in inner_cv.split(train_val_idx):



            train_sessions = train_val_idx[inner_train_idx]
            val_sessions = train_val_idx[inner_val_idx]

            ds_train, ds_val, _ = format_into_datasets_by_index(
                xs_all, ys_all,
                rnn_utils.DatasetRNN,
                train_idx=train_sessions,
                val_idx=val_sessions,
                test_idx=[],
                batch_size=bs
            )

            use_hidden_state = 'False'
            use_previous_values = 'True'
            fit_forget = "False"
            habit_weight = 1

            value_weight = 1.  # This is needed for it to be doing RL

            rnn_rl_params = {
                's': use_hidden_state == 'True',
                'o': use_previous_values == 'True',
                'fit_forget': fit_forget == 'True',
                'forget': 0.,
                'w_h': habit_weight,
                'w_v': value_weight}
            network_params = {'n_actions': 2, 'hidden_size': hs}

            def make_conann():
              model = hybrnn.BiRNN(rl_params=rnn_rl_params, network_params=network_params)
              return model

            params, _ = rnn_utils.fit_model(
                model_fun=make_conann,
                dataset_train=ds_train,
                dataset_test=ds_val,
                loss_fun='categorical',
                optimizer=optax.chain(
                    optax.add_decayed_weights(wd),
                    optax.adam(learning_rate=lr)
                ),
                n_steps_per_call=1000,
                n_steps_max=100000,
                early_stop_step=200,
                if_early_stop=True
            )

            avg_nll = evaluate_nll_over_full_dataset(ds_val, make_conann, params)
            inner_scores.append(avg_nll)

        mean_inner_score = np.mean(inner_scores)
        df_inner_existing = pd.concat([
            df_inner_existing,
            pd.DataFrame([{
                'outer_fold': outer_fold,
                'hidden_size': hs, 'lr': lr, 'weight_decay': wd, 'batch_size': bs,
                'avg_val_nll': mean_inner_score,
                'time': time.time() - t0
            }])
        ], ignore_index=True)
        df_inner_existing.to_csv(inner_csv, index=False)

        if mean_inner_score < best_score:
            best_score = mean_inner_score
            best_params = {'hidden_size': hs, 'lr': lr, 'weight_decay': wd, 'batch_size': bs}

    print(f"Best params for outer fold {outer_fold}: {best_params}, val NLL={best_score:.4f}")

    # ==================
    # 外层训练 + 测试
    # ==================
    ds_train_val, _, ds_test = format_into_datasets_by_index(
        xs_all, ys_all,
        rnn_utils.DatasetRNN,
        train_idx=train_val_idx,
        val_idx=[],
        test_idx=test_idx,
        batch_size=best_params['batch_size']
    )

    use_hidden_state = 'False'
    use_previous_values = 'True'
    fit_forget = "False"
    habit_weight = 1

    value_weight = 1.  # This is needed for it to be doing RL

    rnn_rl_params = {
        's': use_hidden_state == 'True',
        'o': use_previous_values == 'True',
        'fit_forget': fit_forget == 'True',
        'forget': 0.,
        'w_h': habit_weight,
        'w_v': value_weight}
    network_params = {'n_actions': 2, 'hidden_size': best_params['hidden_size']}

    def make_best_conann():
      model = hybrnn.BiRNN(rl_params=rnn_rl_params, network_params=network_params)
      return model

    params, _ = rnn_utils.fit_model(
        model_fun=make_best_conann,
        dataset_train=ds_train_val,
        dataset_test=ds_test,
        loss_fun='categorical',
        optimizer=optax.chain(
            optax.add_decayed_weights(best_params['weight_decay']),
            optax.adam(learning_rate=best_params['lr'])
        ),
        n_steps_per_call=1000,
        n_steps_max=100000,
        early_stop_step=200,
        if_early_stop=True
    )

    test_nll = evaluate_nll_over_full_dataset(ds_test, make_best_conann, params)
    print(f"Outer fold {outer_fold} test NLL: {test_nll:.4f}")

    df_outer_existing = pd.concat([
        df_outer_existing,
        pd.DataFrame([{
            'outer_fold': outer_fold,
            'best_hidden_size': best_params['hidden_size'],
            'best_lr': best_params['lr'],
            'best_weight_decay': best_params['weight_decay'],
            'best_batch_size': best_params['batch_size'],
            'test_nll': test_nll
        }])
    ], ignore_index=True)
    df_outer_existing.to_csv(outer_csv, index=False)

# =====================================
# Step 5: 最终结果
# =====================================
avg_nll = df_outer_existing['test_nll'].mean()
se_nll = df_outer_existing['test_nll'].std(ddof=1) / np.sqrt(len(df_outer_existing))
print(f"\nFinal Average NLL: {avg_nll:.4f} ± {se_nll:.4f}")


Streaming output truncated to the last 5000 lines.
[All Batches Averaged] NLL: 0.4162
Step 1000 of 1000; Loss: 2.0516e+03; Test Loss: 1.8502e+03. (Time: 3.1s)
first loss: inf test loss new: 2173.5528784179687
updating best model ..
Step 1000 of 100000; Loss: 2.1736e+03. (Time: 0.0s)
Step 1000 of 1000; Loss: 9.9425e+02; Test Loss: 1.8503e+03. (Time: 3.2s)
first loss: 2173.5528784179687 test loss new: 1588.0962243652343
updating best model ..
Step 2000 of 100000; Loss: 1.5881e+03. (Time: 0.0s)
Step 1000 of 1000; Loss: 1.8809e+03; Test Loss: 2.0820e+03. (Time: 3.1s)
first loss: 1588.0962243652343 test loss new: 1490.4083276367187
updating best model ..
Step 3000 of 100000; Loss: 1.4904e+03. (Time: 0.0s)
Step 1000 of 1000; Loss: 9.1961e+02; Test Loss: 1.8377e+03. (Time: 3.1s)
first loss: 1490.4083276367187 test loss new: 1427.0273767089843
updating best model ..
Step 4000 of 100000; Loss: 1.4270e+03. (Time: 0.0s)
Step 1000 of 1000; Loss: 1.7456e+03; Test Loss: 7.9509e+02. (Time: 3.2s)
firs

In [ ]:
import numpy as np
import itertools
import os
import pandas as pd
import time
from sklearn.model_selection import KFold

# =====================================
# Step 0: 合并所有实验到 session 维
# =====================================
# 假设 xs_human_list / ys_human_list: list of 3 arrays
# 每个数组形状: (n_trials, n_sessions_exp, feature_dim)
xs_all = np.concatenate(xs_human_list, axis=1)  # 在 session 维拼接
ys_all = np.concatenate(ys_human_list, axis=1)  # 同样在 session 维拼接

n_trials, total_sessions, feature_dim = xs_all.shape
_, _, target_dim = ys_all.shape

print(f"总 trials: {n_trials}, 总 sessions: {total_sessions}")
print(f"X shape: {xs_all.shape}, Y shape: {ys_all.shape}")

# =====================================
# Step 1: 按索引切分 dataset
# =====================================
def format_into_datasets_by_index(xs,
                                  ys,
                                  dataset_constructor,
                                  train_idx,
                                  val_idx,
                                  test_idx,
                                  batch_size=None):
    ds_train = dataset_constructor(xs[:, train_idx], ys[:, train_idx], batch_size=batch_size)
    ds_val   = dataset_constructor(xs[:, val_idx], ys[:, val_idx], batch_size=batch_size)
    ds_test  = dataset_constructor(xs[:, test_idx], ys[:, test_idx], batch_size=batch_size)
    return ds_train, ds_val, ds_test

# =====================================
# Step 2: 超参数网格
# =====================================
param_grid = {
    'hidden_size': [8, 16, 32, 64],
    'lr': [1e-3, 1e-4],
    'weight_decay': [1e-3, 1e-4],
    'batch_size': [32, 64],
}
K_outer = 10
K_inner = 3
random_seed = 42

# =====================================
# Step 3: 结果保存路径
# =====================================
OUTPUT_DIR = '/content/drive/MyDrive/Results'
os.makedirs(OUTPUT_DIR, exist_ok=True)
inner_csv = os.path.join(OUTPUT_DIR, 'human_memann_inner_search_results_sub1.csv')
outer_csv = os.path.join(OUTPUT_DIR, 'human_memann_outer_results_sub1.csv')

if os.path.exists(inner_csv):
    df_inner_existing = pd.read_csv(inner_csv)
else:
    df_inner_existing = pd.DataFrame(columns=[
        'outer_fold', 'hidden_size', 'lr', 'weight_decay', 'batch_size', 'avg_val_nll', 'time'
    ])
if os.path.exists(outer_csv):
    df_outer_existing = pd.read_csv(outer_csv)
else:
    df_outer_existing = pd.DataFrame(columns=[
        'outer_fold', 'best_hidden_size', 'best_lr', 'best_weight_decay', 'best_batch_size', 'test_nll'
    ])

# =====================================
# Step 4: 外层 CV
# =====================================
outer_cv = KFold(n_splits=K_outer, shuffle=True, random_state=random_seed)
session_indices = np.arange(total_sessions)

for outer_fold, (train_val_idx, test_idx) in enumerate(outer_cv.split(session_indices), start=1):

    print(f"\n=== Outer Fold {outer_fold}/{K_outer} ===")

    if outer_fold not in AS_OUTER_LOOP:
        print(f"Skipping outer fold {outer_fold} (not assigned in this notebook)")
        # print('train_val_idx: ', train_val_idx)
        print('test_idx: ', test_idx)
        continue

    if (df_outer_existing['outer_fold'] == outer_fold).any():
        print(f"Skipping outer fold {outer_fold} (already done)")
        continue

    print('test_idx: ', test_idx)
    # ==================
    # 内层超参搜索
    # ==================
    inner_cv = KFold(n_splits=K_inner, shuffle=True, random_state=random_seed)
    best_score = float('inf')
    best_params = None

    for hs, lr, wd, bs in itertools.product(
            param_grid['hidden_size'],
            param_grid['lr'],
            param_grid['weight_decay'],
            param_grid['batch_size']):

        mask = (
            (df_inner_existing['outer_fold'] == outer_fold) &
            (df_inner_existing['hidden_size'] == hs) &
            (df_inner_existing['lr'] == lr) &
            (df_inner_existing['weight_decay'] == wd) &
            (df_inner_existing['batch_size'] == bs)
        )
        if mask.any():
            print(f"Skipping inner config fold {outer_fold}: hs={hs}, lr={lr}, wd={wd}, bs={bs}")
            continue

        inner_scores = []
        t0 = time.time()

        for inner_train_idx, inner_val_idx in inner_cv.split(train_val_idx):



            train_sessions = train_val_idx[inner_train_idx]
            val_sessions = train_val_idx[inner_val_idx]

            ds_train, ds_val, _ = format_into_datasets_by_index(
                xs_all, ys_all,
                rnn_utils.DatasetRNN,
                train_idx=train_sessions,
                val_idx=val_sessions,
                test_idx=[],
                batch_size=bs
            )

            use_hidden_state = 'True'
            use_previous_values = 'False'
            fit_forget = "False"
            habit_weight = 1

            value_weight = 1.  # This is needed for it to be doing RL

            rnn_rl_params = {
                's': use_hidden_state == 'True',
                'o': use_previous_values == 'True',
                'fit_forget': fit_forget == 'True',
                'forget': 0.,
                'w_h': habit_weight,
                'w_v': value_weight}
            network_params = {'n_actions': 2, 'hidden_size': hs}

            def make_memann():
              model = hybrnn.BiRNN(rl_params=rnn_rl_params, network_params=network_params)
              return model

            params, _ = rnn_utils.fit_model(
                model_fun=make_memann,
                dataset_train=ds_train,
                dataset_test=ds_val,
                loss_fun='categorical',
                optimizer=optax.chain(
                    optax.add_decayed_weights(wd),
                    optax.adam(learning_rate=lr)
                ),
                n_steps_per_call=1000,
                n_steps_max=100000,
                early_stop_step=200,
                if_early_stop=True
            )

            avg_nll = evaluate_nll_over_full_dataset(ds_val, make_memann, params)
            inner_scores.append(avg_nll)

        mean_inner_score = np.mean(inner_scores)
        df_inner_existing = pd.concat([
            df_inner_existing,
            pd.DataFrame([{
                'outer_fold': outer_fold,
                'hidden_size': hs, 'lr': lr, 'weight_decay': wd, 'batch_size': bs,
                'avg_val_nll': mean_inner_score,
                'time': time.time() - t0
            }])
        ], ignore_index=True)
        df_inner_existing.to_csv(inner_csv, index=False)

        if mean_inner_score < best_score:
            best_score = mean_inner_score
            best_params = {'hidden_size': hs, 'lr': lr, 'weight_decay': wd, 'batch_size': bs}

    print(f"Best params for outer fold {outer_fold}: {best_params}, val NLL={best_score:.4f}")

    # ==================
    # 外层训练 + 测试
    # ==================
    ds_train_val, _, ds_test = format_into_datasets_by_index(
        xs_all, ys_all,
        rnn_utils.DatasetRNN,
        train_idx=train_val_idx,
        val_idx=[],
        test_idx=test_idx,
        batch_size=best_params['batch_size']
    )

    use_hidden_state = 'True'
    use_previous_values = 'False'
    fit_forget = "False"
    habit_weight = 1

    value_weight = 1.  # This is needed for it to be doing RL

    rnn_rl_params = {
        's': use_hidden_state == 'True',
        'o': use_previous_values == 'True',
        'fit_forget': fit_forget == 'True',
        'forget': 0.,
        'w_h': habit_weight,
        'w_v': value_weight}
    network_params = {'n_actions': 2, 'hidden_size': best_params['hidden_size']}

    def make_best_memann():
      model = hybrnn.BiRNN(rl_params=rnn_rl_params, network_params=network_params)
      return model

    params, _ = rnn_utils.fit_model(
        model_fun=make_best_memann,
        dataset_train=ds_train_val,
        dataset_test=ds_test,
        loss_fun='categorical',
        optimizer=optax.chain(
            optax.add_decayed_weights(best_params['weight_decay']),
            optax.adam(learning_rate=best_params['lr'])
        ),
        n_steps_per_call=1000,
        n_steps_max=100000,
        early_stop_step=200,
        if_early_stop=True
    )

    test_nll = evaluate_nll_over_full_dataset(ds_test, make_best_memann, params)
    print(f"Outer fold {outer_fold} test NLL: {test_nll:.4f}")

    df_outer_existing = pd.concat([
        df_outer_existing,
        pd.DataFrame([{
            'outer_fold': outer_fold,
            'best_hidden_size': best_params['hidden_size'],
            'best_lr': best_params['lr'],
            'best_weight_decay': best_params['weight_decay'],
            'best_batch_size': best_params['batch_size'],
            'test_nll': test_nll
        }])
    ], ignore_index=True)
    df_outer_existing.to_csv(outer_csv, index=False)

# =====================================
# Step 5: 最终结果
# =====================================
avg_nll = df_outer_existing['test_nll'].mean()
se_nll = df_outer_existing['test_nll'].std(ddof=1) / np.sqrt(len(df_outer_existing))
print(f"\nFinal Average NLL: {avg_nll:.4f} ± {se_nll:.4f}")
